In [1]:
#Importing Relevant Files
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pandas as pd
from sklearn import preprocessing

from xgboost import plot_importance
import numpy as np
from sklearn.metrics import confusion_matrix


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
#Reading the Training Dataset
df = pd.read_csv("train.csv")
df

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
0,1,2021-01-01,1,2,2.0,2020-09-24,0,0,0,0,0,0,1,1,0,0,0,0,0
1,2,2021-01-01,2,1,2.0,2020-09-19,1,0,1,0,0,0,1,0,0,0,0,0,0
2,3,2021-01-01,9,3,3.0,2021-08-11,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,2021-01-01,6,7,2.0,2017-10-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,2021-01-01,4,6,NaN,2020-06-08,0,0,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39156,39157,2021-12-31,11,11,2.0,2017-10-19,1,0,0,0,1,1,1,0,0,0,0,0,0
39157,39158,2021-12-31,3,9,3.0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
39158,39159,2021-12-31,8,7,2.0,NaN,1,0,0,0,1,0,1,0,0,0,0,0,0
39159,39160,2021-12-31,7,12,2.0,NaN,0,0,0,0,0,1,0,0,0,0,1,0,0


In [3]:
#Reading the Test/Validation Dataset
dfvalidation = pd.read_csv("test.csv")
dfvalidation['buy'] = 5

dfvalidation

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
0,39162,2022-01-01,2,2,NaN,2021-08-17,1,1,0,0,0,1,0,0,0,0,1,0,5
1,39163,2022-01-01,4,7,3.0,2020-05-21,1,0,0,0,0,0,0,1,0,0,0,0,5
2,39164,2022-01-01,8,7,NaN,NaN,0,0,0,0,1,1,0,0,0,0,0,0,5
3,39165,2022-01-01,9,8,2.0,2020-06-22,0,0,0,0,1,1,1,0,0,0,2,0,5
4,39166,2022-01-01,4,5,2.0,2021-03-10,1,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,52341,2022-03-30,16,12,NaN,NaN,0,0,0,0,0,1,1,0,0,0,0,0,5
13180,52342,2022-03-30,12,8,NaN,2022-03-19,1,0,0,0,0,1,0,0,0,0,0,0,5
13181,52343,2022-03-30,14,14,3.0,2021-08-15,0,0,1,0,0,0,1,0,0,0,0,0,5
13182,52344,2022-03-30,14,10,NaN,NaN,0,0,0,0,0,0,1,0,0,0,1,0,5


In [4]:
##Merging the train and test dataset
dfmerged = pd.concat([df,dfvalidation])
dfmerged.shape

(52345, 19)

In [5]:
#Information about the Merged Dataset
dfmerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52345 entries, 0 to 13183
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    52345 non-null  int64  
 1   created_at            52345 non-null  object 
 2   campaign_var_1        52345 non-null  int64  
 3   campaign_var_2        52345 non-null  int64  
 4   products_purchased    23298 non-null  float64
 5   signup_date           30583 non-null  object 
 6   user_activity_var_1   52345 non-null  int64  
 7   user_activity_var_2   52345 non-null  int64  
 8   user_activity_var_3   52345 non-null  int64  
 9   user_activity_var_4   52345 non-null  int64  
 10  user_activity_var_5   52345 non-null  int64  
 11  user_activity_var_6   52345 non-null  int64  
 12  user_activity_var_7   52345 non-null  int64  
 13  user_activity_var_8   52345 non-null  int64  
 14  user_activity_var_9   52345 non-null  int64  
 15  user_activity_var_1

In [6]:
#Descritpitve Statistics about the Merged Dataset
dfmerged.describe()

,id,campaign_var_1,campaign_var_2,products_purchased,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
count,52345.000000,52345.000000,52345.000000,23298.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000,52345.000000
mean,26173.000000,6.826211,6.828198,2.144347,0.396714,0.006094,0.102952,0.010373,0.151896,0.506295,0.291107,0.166434,0.010679,0.000344,0.227624,0.000478,1.297507
std,15110.844257,3.543873,2.711333,0.780816,0.508333,0.077828,0.303899,0.101322,0.359933,0.560053,0.457671,0.372679,0.102973,0.018541,0.439849,0.021849,2.156715
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13087.000000,4.000000,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26173.000000,6.000000,7.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,39259.000000,9.000000,9.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
max,52345.000000,16.000000,15.000000,4.000000,3.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,2.000000,2.000000,1.000000,4.000000,1.000000,5.000000


In [7]:
dfmerged.mode()

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
0,1,2021-12-26,6.0,7.0,2.0,2022-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52340,52341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52341,52342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52342,52343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52343,52344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Filling Missing Entries with maximum occuring event
dfmerged['signup_date'].fillna('2022-03-06', inplace=True)
dfmerged['products_purchased'].fillna('2.0', inplace=True)
dfmerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52345 entries, 0 to 13183
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    52345 non-null  int64 
 1   created_at            52345 non-null  object
 2   campaign_var_1        52345 non-null  int64 
 3   campaign_var_2        52345 non-null  int64 
 4   products_purchased    52345 non-null  object
 5   signup_date           52345 non-null  object
 6   user_activity_var_1   52345 non-null  int64 
 7   user_activity_var_2   52345 non-null  int64 
 8   user_activity_var_3   52345 non-null  int64 
 9   user_activity_var_4   52345 non-null  int64 
 10  user_activity_var_5   52345 non-null  int64 
 11  user_activity_var_6   52345 non-null  int64 
 12  user_activity_var_7   52345 non-null  int64 
 13  user_activity_var_8   52345 non-null  int64 
 14  user_activity_var_9   52345 non-null  int64 
 15  user_activity_var_10  52345 non-null

In [9]:
dfmerged.isnull().sum()

id                      0
created_at              0
campaign_var_1          0
campaign_var_2          0
products_purchased      0
signup_date             0
user_activity_var_1     0
user_activity_var_2     0
user_activity_var_3     0
user_activity_var_4     0
user_activity_var_5     0
user_activity_var_6     0
user_activity_var_7     0
user_activity_var_8     0
user_activity_var_9     0
user_activity_var_10    0
user_activity_var_11    0
user_activity_var_12    0
buy                     0
dtype: int64

In [10]:
#Creating features from 2 date columns
dfmerged['created_at'] = pd.to_datetime(dfmerged['created_at'], format = "%Y-%m-%d")
dfmerged['signup_date'] = pd.to_datetime(dfmerged['signup_date'], format = "%Y-%m-%d")
dfmerged['created_at_years'] = dfmerged['created_at'].dt.year
dfmerged['created_at_month'] = dfmerged['created_at'].dt.month
dfmerged['created_at_week'] = dfmerged['created_at'].dt.week
dfmerged['created_at_day'] = dfmerged['created_at'].dt.day
dfmerged['created_at_dayofweek'] = dfmerged['created_at'].dt.dayofweek

dfmerged['signup_date_year'] = dfmerged['signup_date'].dt.year
dfmerged['signup_date_month'] = dfmerged['signup_date'].dt.month
dfmerged['signup_date_week'] = dfmerged['signup_date'].dt.week
dfmerged['signup_date_at_day'] = dfmerged['signup_date'].dt.day
dfmerged['signup_date_at_dayofweek'] = dfmerged['signup_date'].dt.dayofweek

dfmerged

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if sys.path[0] == '':


,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy,created_at_years,created_at_month,created_at_week,created_at_day,created_at_dayofweek,signup_date_year,signup_date_month,signup_date_week,signup_date_at_day,signup_date_at_dayofweek
0,1,2021-01-01,1,2,2.0,2020-09-24,0,0,0,0,0,0,1,1,0,0,0,0,0,2021,1,53,1,4,2020,9,39,24,3
1,2,2021-01-01,2,1,2.0,2020-09-19,1,0,1,0,0,0,1,0,0,0,0,0,0,2021,1,53,1,4,2020,9,38,19,5
2,3,2021-01-01,9,3,3.0,2021-08-11,1,0,0,0,0,0,0,0,0,0,0,0,0,2021,1,53,1,4,2021,8,32,11,2
3,4,2021-01-01,6,7,2.0,2017-10-04,0,0,0,0,0,0,0,0,0,0,0,0,0,2021,1,53,1,4,2017,10,40,4,2
4,5,2021-01-01,4,6,2.0,2020-06-08,0,0,0,0,0,0,1,0,0,0,1,0,0,2021,1,53,1,4,2020,6,24,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,52341,2022-03-30,16,12,2.0,2022-03-06,0,0,0,0,0,1,1,0,0,0,0,0,5,2022,3,13,30,2,2022,3,9,6,6
13180,52342,2022-03-30,12,8,2.0,2022-03-19,1,0,0,0,0,1,0,0,0,0,0,0,5,2022,3,13,30,2,2022,3,11,19,5
13181,52343,2022-03-30,14,14,3.0,2021-08-15,0,0,1,0,0,0,1,0,0,0,0,0,5,2022,3,13,30,2,2021,8,32,15,6
13182,52344,2022-03-30,14,10,2.0,2022-03-06,0,0,0,0,0,0,1,0,0,0,1,0,5,2022,3,13,30,2,2022,3,9,6,6


In [11]:
dfmerged = pd.concat([dfmerged[['products_purchased','buy','created_at_week','created_at_day','signup_date_week','signup_date_at_day']],
                      pd.get_dummies(dfmerged['campaign_var_1'],drop_first = True, prefix=['col1']),pd.get_dummies(dfmerged['campaign_var_2'],drop_first = True, prefix=['col2']),
                      pd.get_dummies(dfmerged['user_activity_var_1'],drop_first = True, prefix=['col3']),pd.get_dummies(dfmerged['user_activity_var_2'],drop_first = True, prefix=['col4']),
                      pd.get_dummies(dfmerged['user_activity_var_3'],drop_first = True, prefix=['col5']),pd.get_dummies(dfmerged['user_activity_var_4'],drop_first = True, prefix=['col6']),
                      pd.get_dummies(dfmerged['user_activity_var_5'],drop_first = True, prefix=['col7']),pd.get_dummies(dfmerged['user_activity_var_6'],drop_first = True, prefix=['col8']),
                      pd.get_dummies(dfmerged['user_activity_var_7'],drop_first = True, prefix=['col9']),pd.get_dummies(dfmerged['user_activity_var_8'],drop_first = True, prefix=['col10']),
                      pd.get_dummies(dfmerged['user_activity_var_9'],drop_first = True, prefix=['col11']),pd.get_dummies(dfmerged['user_activity_var_10'],drop_first = True, prefix=['col12']),
                      pd.get_dummies(dfmerged['user_activity_var_11'],drop_first = True, prefix=['col13']),pd.get_dummies(dfmerged['user_activity_var_12'],drop_first = True, prefix=['col14']),
                      pd.get_dummies(dfmerged['created_at_years'],drop_first = True, prefix=['col15']),pd.get_dummies(dfmerged['created_at_month'],drop_first = True, prefix=['col16']),
                      pd.get_dummies(dfmerged['created_at_dayofweek'],drop_first = True, prefix=['col19']),pd.get_dummies(dfmerged['signup_date_year'],drop_first = True, prefix=['col20']),
                      pd.get_dummies(dfmerged['signup_date_month'],drop_first = True, prefix=['col21']),pd.get_dummies(dfmerged['signup_date_at_dayofweek'],drop_first = True, prefix=['col24'])],axis=1)

dfmerged

,products_purchased,buy,created_at_week,created_at_day,signup_date_week,signup_date_at_day,['col1']_2,['col1']_3,['col1']_4,['col1']_5,['col1']_6,['col1']_7,['col1']_8,['col1']_9,['col1']_10,['col1']_11,['col1']_12,['col1']_13,['col1']_14,['col1']_15,['col1']_16,['col2']_2,['col2']_3,['col2']_4,['col2']_5,['col2']_6,['col2']_7,['col2']_8,['col2']_9,['col2']_10,['col2']_11,['col2']_12,['col2']_13,['col2']_14,['col2']_15,['col3']_1,['col3']_2,['col3']_3,['col4']_1,['col5']_1,['col6']_1,['col7']_1,['col7']_2,['col8']_1,['col8']_2,['col8']_3,['col9']_1,['col9']_2,['col10']_1,['col10']_2,['col11']_1,['col11']_2,['col12']_1,['col13']_1,['col13']_2,['col13']_3,['col13']_4,['col14']_1,['col15']_2022,['col16']_2,['col16']_3,['col16']_4,['col16']_5,['col16']_6,['col16']_7,['col16']_8,['col16']_9,['col16']_10,['col16']_11,['col16']_12,['col19']_1,['col19']_2,['col19']_3,['col19']_4,['col19']_5,['col19']_6,['col20']_2016,['col20']_2017,['col20']_2018,['col20']_2019,['col20']_2020,['col20']_2021,['col20']_2022,['col21']_2,['col21']_3,['col21']_4,['col21']_5,['col21']_6,['col21']_7,['col21']_8,['col21']_9,['col21']_10,['col21']_11,['col21']_12,['col24']_1,['col24']_2,['col24']_3,['col24']_4,['col24']_5,['col24']_6
0,2.0,0,53,1,39,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,2.0,0,53,1,38,19,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,3.0,0,53,1,32,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,2.0,0,53,1,40,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,2.0,0,53,1,24,8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,2.0,5,13,30,9,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13180,2.0,5,13,30,11,19,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13181,3.0,5,13,30,32,15,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
13182,2.0,5,13,30,9,6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [12]:
###Generating all possible pair of interactions between 2 pair of columns.
##Then removing those columns 

from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

Y = dfmerged.buy
X = dfmerged.drop(['buy'],1)
 


def add_interactions(df1):
    combos = list(combinations(list(df1.columns), 2))
    colnames = list(df1.columns)+['_'.join(x) for x in combos]
    
    #scaler = MinMaxScaler()
    #scaler.fit(df1)
    #df = scaler.transform(df1)
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df1 = poly.fit_transform(df1)
    df1 = pd.DataFrame(df1)
    df1.columns = colnames
    
    noint_indices = [i for i,x in enumerate(list((df1==0).all())) if x]
    df1= df1.drop(df1.columns[noint_indices], axis=1)
    
    return df1
X = add_interactions(X)





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [13]:
X.shape
Y

0        0
1        0
2        0
3        0
4        0
        ..
13179    5
13180    5
13181    5
13182    5
13183    5
Name: buy, Length: 52345, dtype: int64

In [14]:
###Seperate out the training and testing dataset

X.index = Y.index
X['buy'] = Y

X_validation = X[39161:]
X = X[0:39161]


In [15]:
X

,products_purchased,created_at_week,created_at_day,signup_date_week,signup_date_at_day,['col1']_2,['col1']_3,['col1']_4,['col1']_5,['col1']_6,['col1']_7,['col1']_8,['col1']_9,['col1']_10,['col1']_11,['col1']_12,['col1']_13,['col1']_14,['col1']_15,['col1']_16,['col2']_2,['col2']_3,['col2']_4,['col2']_5,['col2']_6,['col2']_7,['col2']_8,['col2']_9,['col2']_10,['col2']_11,['col2']_12,['col2']_13,['col2']_14,['col2']_15,['col3']_1,['col3']_2,['col3']_3,['col4']_1,['col5']_1,['col6']_1,['col7']_1,['col7']_2,['col8']_1,['col8']_2,['col8']_3,['col9']_1,['col9']_2,['col10']_1,['col10']_2,['col11']_1,['col11']_2,['col12']_1,['col13']_1,['col13']_2,['col13']_3,['col13']_4,['col14']_1,['col15']_2022,['col16']_2,['col16']_3,['col16']_4,['col16']_5,['col16']_6,['col16']_7,['col16']_8,['col16']_9,['col16']_10,['col16']_11,['col16']_12,['col19']_1,['col19']_2,['col19']_3,['col19']_4,['col19']_5,['col19']_6,['col20']_2016,['col20']_2017,['col20']_2018,['col20']_2019,['col20']_2020,['col20']_2021,['col20']_2022,['col21']_2,['col21']_3,['col21']_4,['col21']_5,['col21']_6,['col21']_7,['col21']_8,['col21']_9,['col21']_10,['col21']_11,['col21']_12,['col24']_1,['col24']_2,['col24']_3,['col24']_4,['col24']_5,['col24']_6,products_purchased_created_at_week,products_purchased_created_at_day,products_purchased_signup_date_week,products_purchased_signup_date_at_day,products_purchased_['col1']_2,products_purchased_['col1']_3,products_purchased_['col1']_4,products_purchased_['col1']_5,products_purchased_['col1']_6,products_purchased_['col1']_7,products_purchased_['col1']_8,products_purchased_['col1']_9,products_purchased_['col1']_10,products_purchased_['col1']_11,products_purchased_['col1']_12,products_purchased_['col1']_13,products_purchased_['col1']_14,products_purchased_['col1']_15,products_purchased_['col1']_16,products_purchased_['col2']_2,products_purchased_['col2']_3,products_purchased_['col2']_4,products_purchased_['col2']_5,products_purchased_['col2']_6,products_purchased_['col2']_7,products_purchased_['col2']_8,products_purchased_['col2']_9,products_purchased_['col2']_10,products_purchased_['col2']_11,products_purchased_['col2']_12,products_purchased_['col2']_13,products_purchased_['col2']_14,products_purchased_['col2']_15,products_purchased_['col3']_1,products_purchased_['col3']_2,products_purchased_['col3']_3,products_purchased_['col4']_1,products_purchased_['col5']_1,products_purchased_['col6']_1,products_purchased_['col7']_1,products_purchased_['col7']_2,products_purchased_['col8']_1,products_purchased_['col8']_2,products_purchased_['col8']_3,products_purchased_['col9']_1,products_purchased_['col9']_2,products_purchased_['col10']_1,products_purchased_['col10']_2,products_purchased_['col11']_1,products_purchased_['col11']_2,products_purchased_['col12']_1,products_purchased_['col13']_1,products_purchased_['col13']_2,products_purchased_['col13']_3,products_purchased_['col13']_4,products_purchased_['col14']_1,products_purchased_['col15']_2022,products_purchased_['col16']_2,products_purchased_['col16']_3,products_purchased_['col16']_4,products_purchased_['col16']_5,products_purchased_['col16']_6,products_purchased_['col16']_7,products_purchased_['col16']_8,products_purchased_['col16']_9,products_purchased_['col16']_10,products_purchased_['col16']_11,products_purchased_['col16']_12,products_purchased_['col19']_1,products_purchased_['col19']_2,products_purchased_['col19']_3,products_purchased_['col19']_4,products_purchased_['col19']_5,products_purchased_['col19']_6,products_purchased_['col20']_2016,products_purchased_['col20']_2017,products_purchased_['col20']_2018,products_purchased_['col20']_2019,products_purchased_['col20']_2020,products_purchased_['col20']_2021,products_purchased_['col20']_2022,products_purchased_['col21']_2,products_purchased_['col21']_3,products_purchased_['col21']_4,products_purchased_['col21']_5,products_purchased_['col21']_6,products_purchased_['col21']_7,products_purchased_['col21']_8,products_purchased_['col21']_9,products

In [16]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]
X_validation.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_validation.columns.values]


In [17]:
Y = X.buy
X = X.drop(['buy'],1)
 

seed = 2
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
###Hypertune the model intensively with a 5 fold cross validation strategy.
##first grid search for max_depth,min_child_weight then fix those and search for rest in same fashion
### parameter scale_pos_weight is quite important in case of imbalanced dataset

from sklearn.model_selection import GridSearchCV
param_test2b = {
  'min_child_weight':[5,6],
  'max_depth': range(3,10,2),
  #'n_estimators':[150,200,300,400],
  #'scale_pos_weight':[1,2,3,4],
  #'colsample_bytree':[0.7,0.8], 
  #'subsample':[0.7,0.8],
  #'gamma':[0,0.2.0.4]
    
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=3,seed=27), 
 param_grid = param_test2b, scoring='f1',n_jobs=4, cv=5)
gsearch2b.fit(X_train, y_train)
print(gsearch2b.grid_scores_)
print("gsearch2b.best_params_",gsearch2b.best_params_)
print("gsearch2b.best_score_",gsearch2b.best_score_)

In [19]:
###Train the model with the best params
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)
modelXg.fit(X_train, y_train)
y_xg = modelXg.predict(X_test)

In [21]:
## Determine whether your model is overfitting or not , with the help of ROC.
print(confusion_matrix(y_test, y_xg))
predictions = [value for value in y_xg]
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
print("roc_auc test set", roc_auc_score(y_test, modelXg.predict_proba(X_test)[:,1]))
print("roc_auc training set", roc_auc_score(y_train, modelXg.predict_proba(X_train)[:,1]))

[[7378   61]
 [ 157  237]]
Accuracy_score: 97.22% on test dataset
precision_score: 79.53% on test dataset
recall_score: 60.15% on test dataset
f1_score: 68.50% on test dataset
roc_auc test set 0.9352568402362909
roc_auc training set 0.9786451500940825


In [22]:
##Once the model is ready, train the model on entire dataset.
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)

modelXg.fit(X, Y)


XGBClassifier(colsample_bytree=0.8, gamma=0.4, max_depth=4, min_child_weight=7,
              n_estimators=200, nthread=4, scale_pos_weight=3, seed=29,
              subsample=0.8)

In [23]:
X = X_validation.drop(['buy'],1)
dfvalidation = X_validation.drop(['buy'],1)
y_valid = modelXg.predict(dfvalidation)
submission = dfvalidation.copy()
submission['buy'] = y_valid
submission['buy'].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


0    12813
1      371
Name: buy, dtype: int64

In [24]:
submission = submission[['buy']]
submission.reset_index(inplace=True)


In [32]:
submission2 = pd.read_csv("test.csv")
submission['id'] = submission2['id']
submission
submission = submission[['id','buy']]
submission
submission.to_csv("output_v6.csv",index=False)